# WebScraping de los resultados históricos de la Formula 1

El objetivo es recopilar en un dataset los datos básicos de todas las carreras puntuables de la Formula 1.
La Formula 1 es un deporte que se viene realizando formalmente desde 1050, donde las escuderías y los pilotos van mejorando continuamente. Cada año se realizan varias carreras en diferentes circuitos, habitualmente en diferentes países. 
Vamos a recopilar información de los resultados de la carrera puntuable para el campeonato, obteniendo la información básica sobre el circuito, el piloto, la escudería y el resultado.

In [1]:
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import requests
import random
import time
import csv

In [2]:
# Raíz de la página sobre la que trabajaremos
url_root = "http://www.formula1.com"
# Página específica donde iniciaremos nuestra extracción de información
url_start = "https://www.formula1.com/en/results.html"

In [3]:
# Creamos un dataset vacio con todas las columnas que vamos a obtener
df1 = pd.DataFrame(columns=['date', 'country','circuit','position'], dtype=str)
df2 = pd.DataFrame(columns=['car_num'], dtype=int)
df3 = pd.DataFrame(columns=['name','surname','alias', 'team'], dtype=str)
df4 = pd.DataFrame(columns=['laps'], dtype=int)
df5 = pd.DataFrame(columns=['duration'], dtype=str)
df6 = pd.DataFrame(columns=['points'], dtype=int)
Formula_1 = pd.concat([df1, df2, df3, df4, df5, df6], axis=1)

Vamos a generar tres funciones diferentes, que son las que nos realizarán la conexión con la página web, verificarán si dicha conexión es correcta, y en caso afirmativo extraerán la información solicitada. 
La primera función extraerá la información de las url de cada año que ha habido en la Formula 1.
La segunda función extraerá la información de las url de cada Gran premio de un año concreto.
La tercera función extraerá la información de detalle de cada carrera carrera específica.

In [4]:
# Función extracción url relativos a los diferentes años de los que hay datos.

def F1_year_extract(link):

    # Incorporamos un retraso de entre 0.5 segundos hasta 2 segundos después de cada obtención de datos
    sleepTimes = [0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 2]
    time.sleep(random.choice(sleepTimes))

    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)
    
    # Verificamos si la petición ha sido correcta, revisando el código que nos ha devuelto la petición.
    if connection.status_code == 200:
   
        # Descargamos la página raíz de los resultados
        soup = BeautifulSoup(connection.content, "lxml")
        
        # Creamos una lista vacía para alojar las url de cada año
        years_url_list = []        
        
        # Acotamos al código donde se especifican las url de cada año
        cod_url_years = soup.find('div', {'class': 'resultsarchive-filter-container'})
    
        # Buscamos cada atributo "li" que tiene el año definido como texto.
        # Extraemos la url relativa, construimos la url completa y la guardamos.
        for tag in cod_url_years.find_all("li"):
            if tag.text.strip('\n').isdigit():
                link = tag.find("a").get("href")
                url = "%s%s" % (url_root, link)
                years_url_list.append(url)  
                
        return years_url_list
    else:
        print("Error de carga en la página inicial:",link)

In [5]:
# Función extracción url relativos a cada gran premio de cada año.

def F1_prix_extract(link):
    
    # Incorporamos un retraso de entre 0.5 segundos hasta 2 segundos después de cada obtención de datos
    sleepTimes = [0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 2]
    time.sleep(random.choice(sleepTimes))
    
    # Realizamos la petición a la url específica de un año concreto.
    connection = requests.get(link)
    
    # Verificamos si la petición ha sido correcta, revisando el código que nos ha devuelto la petición.
    if connection.status_code == 200:
        
        # Creamos una lista vacía para alojar la información de cada Gran Premio de Formula 1
        prix_url_list = []
        
        # Descargamos la página de cada año
        soup = BeautifulSoup(connection.content, "lxml")
        
        # Seleccionamos la división de donde extreremos los links de los eventos.
        table = soup.find('table', {'class':'resultsarchive-table'})
        
        for event in table.tbody.find_all("tr"):
            country = event.a.text.strip('\n ')
            date = event.select('td')[2].text
            link = event.a['href']
            url = "%s%s" % (url_root, link)
            prix_url_list.append((country,date,url))
       
        return(prix_url_list)
      
    else:
        print("Error de carga en la página de un año concreto:",link)


In [6]:
# Función extracción información de resultados para cada gran premio.

def F1_data_extract(link):

    # Incorporamos un retraso de entre 0.5 segundos hasta 2 segundos después de cada obtención de datos
    sleepTimes = [0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 2]
    time.sleep(random.choice(sleepTimes))
    
    # Realizamos la petición a la web de la Fórmula 1 y revisamos la codificación que tiene.
    connection = requests.get(link)
    http_encoding = connection.encoding if 'charset' in connection.headers.get('content-type', '').lower() else None
    html_encoding = EncodingDetector.find_declared_encoding(connection.content, is_html=True)
    encoding = html_encoding or http_encoding
    
    # Verificamos si la petición ha sido correcta, revisando el código que nos ha devuelto la petición.
    if connection.status_code == 200:
        
        # Descargamos la página de cada evento
        soup = BeautifulSoup(connection.content, "lxml", from_encoding=encoding)
        
        # Acotamos al código donde se especifica el nombre del circuito
        circuit = soup.select('span.circuit-info')[0].text
        
        # Seleccionamos la tabla donde extraemos la información.
        table = soup.find('table', {'class':'resultsarchive-table'})
        
        # Creamos una lista donde guardaremos la tupla de datos que obtengamos.
        records = []
        
        # Recorremos la tabla fila a fila extrayendo la información que nos interesa
        for event in table.tbody.find_all("tr"):
            pos = event.select('td')[1].text
            no = event.select('td')[2].text
            name = event.select('span')[0].text
            surname = event.select('span')[1].text
            alias = event.select('span')[2].text
            team = event.select('td')[4].text
            laps = event.select('td')[5].text
            duration = event.select('td')[6].text
            points = event.select('td')[7].text
            records.append((circuit,pos,no,name,surname,alias,team,laps,duration,points))
             

        return(records)

    else:
        print("Error de carga en la página de un evento concreto:",link)

A partir de este punto está el programa que irá utilizando las funciones creadas para la recolección de informacion, y la icorporará en el dataset. Finalmente escribimos el dataset obtenido en un fichero csv.

In [7]:
# Obtenemos el listado de url de cada año.
F1_url_by_year = F1_year_extract(url_start)


In [8]:
# Obtenemos el listado de urls de cada gran premio según el año.
F1_event_url_list = []

# Vamos a extraer la información para cada url de cada año
for season in tqdm(F1_url_by_year):
    
    # Extraemos la lista de url de eventos por cada año
    url_event_list = F1_prix_extract(season)
    
    # Extraemos los datos de la lista para crear una única lista con la información que nos interesa.
    for event in url_event_list:
        country = event[0]
        date = event[1]
        link_e = event[2]
        reg = (country,date,link_e)
        
        # Esta lista contendrá todas las url de todos los eventos.
        F1_event_url_list.append(reg)

100%|██████████| 71/71 [01:53<00:00,  1.60s/it]


In [9]:
# Obtenemos los resultados de cada carrera

#for race in F1_event_url_list:
for race in tqdm(F1_event_url_list):
    
    # Identificamos la información que tenemos en cada elemento de la lista
    country = race[0]
    date = race[1]
    link = race[2]
    
    # Extraemos los datos de cada carrera
    data = F1_data_extract(link)
    
    #recorrer la lista data de los resultado de ese evento concreto
    for result in data:
        
        date_f = datetime.strptime(date, '%d %b %Y').date()
        
        # Incorporar en el dataset cada registro.
        new_reg = {'date':date_f,
                   'country':country, 
                   'circuit':result[0],
                   'position':result[1],
                   'car_num':int(result[2]),
                   'name':result[3],
                   'surname':result[4],
                   'alias':result[5],
                   'team':result[6],
                   'laps':result[7],
                   'duration':result[8],
                   'points':float(result[9])
                  }
        Formula_1.loc[len(Formula_1)] = new_reg
        
    

100%|██████████| 1034/1034 [31:48<00:00,  1.85s/it]


In [10]:
# Escribimos el dataset obtenido en un fichero csv
Formula_1.to_csv('Formula_1_historical_results.csv', index=False, encoding='utf-8')